In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
models = [
         KNeighborsClassifier(3),
         SVC(),
         DecisionTreeClassifier(),
         RandomForestClassifier(),
         ]

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
training = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
print(training.shape, test.shape)

(891, 12) (418, 11)


In [4]:
# Visualizing structure of dataset
# passengerId and name can definitely be dropped 
training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Unique values
training.apply(lambda x: len(x.unique()))

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             89
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          148
Embarked         4
dtype: int64

In [6]:
# Null values
# age has lots of nulls lets see if it is correlated to survived
training.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
# Correlation Matrix
# fare and pclass seem to be only features that correlate with survived
# age had lots of nulls but not correlated anyways
corr = training.corr()
corr

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [8]:
# Dropping irrelevant columns
training = training.drop(columns=['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'])

In [9]:
# Datatype info
# need to change sex to category
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Fare      891 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 28.0+ KB


In [10]:
# Changing sex to category and one-hot encoding categorical features
training['Sex'] = training['Sex'].astype('category')

cols = ['Sex', 'Pclass']

for col in cols:
    training = pd.concat([training, pd.get_dummies(training[col], prefix=col, drop_first=True)], axis=1)
training = training.drop(columns=cols)

In [11]:
# Now to normalize fare
training['Fare'] = (training['Fare'] - training['Fare'].min()) / (training['Fare'].max() - training['Fare'].min())
training.head()

,Survived,Fare,Sex_male,Pclass_2,Pclass_3
0,0,0.014151,1,0,1
1,1,0.139136,0,0,0
2,1,0.015469,0,0,1
3,1,0.103644,0,0,0
4,0,0.015713,1,0,1


In [12]:
# Getting X and y
X = training.drop(columns=['Survived'])
y = training['Survived']

In [13]:
# K Fold cross validation 
def train(model):
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
    pred = model_selection.cross_val_score(model, X, y, cv=kfold, scoring='roc_auc')
    cv_score = pred.mean()
    print(f'Model: {model} CV Score: {cv_score}')

In [14]:
# Comparing models
# RandomForest has higher chance of classifying random positive higher than negative
for model in models:
    train(model)

Model: KNeighborsClassifier(n_neighbors=3) CV Score: 0.8218189346642489
Model: SVC() CV Score: 0.7937604687716957
Model: DecisionTreeClassifier() CV Score: 0.8099357071206892
Model: RandomForestClassifier() CV Score: 0.8429909473191405


In [15]:
# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Random forest clf accuracy
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_pred, y_test)
acc

0.8324022346368715

In [17]:
# KNN accuracy
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_pred, y_test)
acc

0.7821229050279329

In [18]:
# Decision tree clf accuracy
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_pred, y_test)
acc

0.8212290502793296

In [19]:
# Seeing if test has nulls
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [20]:
# P class of null is 3
null = test[test['Fare'].isnull()]
null

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [21]:
# Preparing test df and saving id
submission_df = test.filter(['PassengerId'])
test = test.drop(columns=['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'])
test['Sex'] = test['Sex'].astype('category')

cols = ['Sex', 'Pclass']

for col in cols:
    test = pd.concat([test, pd.get_dummies(test[col], prefix=col, drop_first=True)], axis=1)
test = test.drop(columns=cols)
test['Fare'] = (test['Fare'] - test['Fare'].min()) / (test['Fare'].max() - test['Fare'].min())
test.head()

,Fare,Sex_male,Pclass_2,Pclass_3
0,0.015282,1,0,1
1,0.013663,0,0,1
2,0.018909,1,1,0
3,0.016908,1,0,1
4,0.023984,0,0,1


In [23]:
p_class_3 = test.loc[test['Pclass_3'] == 1]
p_class_3.describe()

,Fare,Sex_male,Pclass_2,Pclass_3
count,217.000000,218.000000,218.0,218.0
mean,0.024320,0.669725,0.0,1.0
std,0.021087,0.471395,0.0,0.0
min,0.006189,0.000000,0.0,1.0
25%,0.015127,0.000000,0.0,1.0
50%,0.015412,1.000000,0.0,1.0
75%,0.028107,1.000000,0.0,1.0
max,0.135753,1.000000,0.0,1.0


In [24]:
# Filling 1 null value with mean of P class 3
test['Fare'].fillna(0.24520, inplace=True)

In [25]:
# Choosing Random forest clf
model = RandomForestClassifier()
model.fit(X_train, y_train)
pred = model.predict(test)

In [26]:
submission_df['Survived'] = pred
submission_df.shape

(418, 2)

In [27]:
submission_df.to_csv('gender_submissions.csv', index=False)